+ [x] Edge Loss の調整：w_edge を調整して“線のシャープさ”と“ノイズ”のトレードオフを取ってください（5〜20の範囲で試すと良い）。

+ [ ] Perceptual Loss（VGG）を加えると線の太さや“見た目”が向上することがあります（ただし学習が複雑に）。torchvision.models.vgg19(pretrained=True) の中間特徴で L1 を取る方式。

+ [ ] PatchGAN 判別器（GAN Loss） を追加すると更にシャープで自然な線が出せます。安定化には History Buffer/LSGAN/PatchGAN が使われます。

+ [ ] Deep Supervision：複数解像度で出力し、それぞれに損失を付けると細線の維持に有効です。

+ [ ] Data augment：線の太さや明度変化、ランダムブラッシュストロークなどを入れると汎化が良くなります。

+ [ ] Inference TTA：左右反転やスケールを平均化して最終出力を滑らかにする。

In [115]:
# train.py
import math
import os
from typing import Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import glob

from torchviz import make_dot

In [116]:
# -----------------------
# Utility blocks
# -----------------------
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size=3, stride=1, padding=1, use_bn=True):
        super().__init__()
        layers = [nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding, bias=not use_bn)]
        if use_bn:
            layers.append(nn.InstanceNorm2d(out_ch, affine=True))
        layers.append(nn.ReLU(inplace=True))
        self.net = nn.Sequential(*layers)
    def forward(self, x): return self.net(x)

class ResidualBlock(nn.Module):
    def __init__(self, ch):
        super().__init__()
        self.conv1 = ConvBlock(ch, ch)
        self.conv2 = nn.Conv2d(ch, ch, 3, 1, 1, bias=False)
        self.norm = nn.InstanceNorm2d(ch, affine=True)
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.norm(out)
        return F.relu(out + x)

# -----------------------
# Simple Attention Gate (spatial attention)
# -----------------------
class SpatialAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=7, padding=3)
    def forward(self, x):
        # channel-wise average and max
        avg = x.mean(dim=1, keepdim=True)
        mx, _ = x.max(dim=1, keepdim=True)
        cat = torch.cat([avg, mx], dim=1)  # 2 x H x W
        # reduce to single-channel attention
        att = torch.sigmoid(self.conv(cat))
        return x * att

# -----------------------
# Basic Self-Attention module (non-local)
# -----------------------
class SelfAttention(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.query_conv = nn.Conv2d(in_dim, in_dim//8, 1)
        self.key_conv   = nn.Conv2d(in_dim, in_dim//8, 1)
        self.value_conv = nn.Conv2d(in_dim, in_dim, 1)
        self.gamma = nn.Parameter(torch.zeros(1))
    def forward(self, x):
        b, c, h, w = x.size()
        proj_query = self.query_conv(x).view(b, -1, h*w).permute(0,2,1)  # B x N x C'
        proj_key   = self.key_conv(x).view(b, -1, h*w)                    # B x C' x N
        energy = torch.bmm(proj_query, proj_key)                          # B x N x N
        att = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(b, -1, h*w)                  # B x C x N
        out = torch.bmm(proj_value, att.permute(0,2,1))                   # B x C x N
        out = out.view(b, c, h, w)
        out = self.gamma * out + x
        return out

# -----------------------
# Sobel (Edge) loss
# -----------------------
class SobelEdgeLoss(nn.Module):
    def __init__(self, p=1):
        super().__init__()
        # Sobel kernels (horizontal, vertical)
        kx = torch.tensor([[1, 0, -1],
                           [2, 0, -2],
                           [1, 0, -1]], dtype=torch.float32) / 4.0
        ky = torch.tensor([[1, 2, 1],
                           [0, 0, 0],
                           [-1, -2, -1]], dtype=torch.float32) / 4.0
        self.register_buffer('kx', kx.view(1,1,3,3))
        self.register_buffer('ky', ky.view(1,1,3,3))
        self.p = p

    def forward(self, pred, target):
        # pred/target: Bx1xHxW
        kx = self.kx.to(device=pred.device, dtype=pred.dtype)
        ky = self.ky.to(device=pred.device, dtype=pred.dtype)

        gx_pred = F.conv2d(pred, kx, padding=1)
        gy_pred = F.conv2d(pred, ky, padding=1)
        gx_tgt  = F.conv2d(target, kx, padding=1)
        gy_tgt  = F.conv2d(target, ky, padding=1)

        edge_pred = torch.sqrt(gx_pred**2 + gy_pred**2 + 1e-6)
        edge_tgt  = torch.sqrt(gx_tgt**2 + gy_tgt**2 + 1e-6)

        if self.p == 1:
            return F.l1_loss(edge_pred, edge_tgt)
        else:
            return F.mse_loss(edge_pred, edge_tgt)


# -----------------------
# Small demo dataset (grayscale pairs)
# Replace with your pair dataset: (rough_sketch, line_art)
# -----------------------
class PairImageDataset(Dataset):
    def __init__(self, rough_dir, line_dir, size=(256,256)):
        super().__init__()
        self.rough_files = sorted(glob.glob(os.path.join(rough_dir, '*.jpg')))
        self.line_files  = sorted(glob.glob(os.path.join(line_dir, '*.jpg')))
        assert len(self.rough_files) == len(self.line_files), "pair counts must match"
        self.tr = transforms.Compose([
            transforms.Resize(size),
            transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor()
        ])
    def __len__(self): return len(self.rough_files)
    def __getitem__(self, idx):
        r = Image.open(self.rough_files[idx]).convert('RGB')
        l = Image.open(self.line_files[idx]).convert('RGB')

        r = self.tr(r)
        l = self.tr(l)

        # --- ここで反転 ---
        #l = 1.0 - l   # ← 白線→黒線に反転

        return r, l



In [122]:

# -----------------------
# U-Net with Residual and Attention
# -----------------------
class AttnResUNet(nn.Module):
    def __init__(self, in_ch=1, out_ch=1, base_ch=64, use_self_attn=True):
        super().__init__()
        self.enc1 = nn.Sequential(ConvBlock(in_ch, base_ch), ResidualBlock(base_ch))
        self.pool1 = nn.MaxPool2d(2)

        self.enc2 = nn.Sequential(ConvBlock(base_ch, base_ch*2), ResidualBlock(base_ch*2))
        self.pool2 = nn.MaxPool2d(2)

        self.enc3 = nn.Sequential(ConvBlock(base_ch*2, base_ch*4), ResidualBlock(base_ch*4))
        self.pool3 = nn.MaxPool2d(2)

        self.bottleneck = nn.Sequential(ConvBlock(base_ch*4, base_ch*8), ResidualBlock(base_ch*8))
        self.self_attn = SelfAttention(base_ch*8) if use_self_attn else nn.Identity()

        # decoder
        self.up3 = nn.ConvTranspose2d(base_ch*8, base_ch*4, 2, 2)
        self.dec3 = nn.Sequential(ConvBlock(base_ch*8, base_ch*4), ResidualBlock(base_ch*4))

        self.up2 = nn.ConvTranspose2d(base_ch*4, base_ch*2, 2, 2)
        self.dec2 = nn.Sequential(ConvBlock(base_ch*4, base_ch*2), ResidualBlock(base_ch*2))

        self.up1 = nn.ConvTranspose2d(base_ch*2, base_ch, 2, 2)
        self.dec1 = nn.Sequential(ConvBlock(base_ch*2, base_ch), ResidualBlock(base_ch))

        self.final = nn.Conv2d(base_ch, out_ch, 1)

        self.spatial_attn1 = SpatialAttention()
        self.spatial_attn2 = SpatialAttention()
        self.spatial_attn3 = SpatialAttention()

    def forward(self, x):
        e1 = self.enc1(x)           # B,64,H,W
        p1 = self.pool1(e1)

        e2 = self.enc2(p1)          # B,128,H/2,W/2
        p2 = self.pool2(e2)

        e3 = self.enc3(p2)          # B,256,H/4,W/4
        p3 = self.pool3(e3)

        b = self.bottleneck(p3)     # B,512,H/8,W/8
        b = self.self_attn(b)

        u3 = self.up3(b)            # B,256,H/4,W/4
        # attention on skip
        #e3_att = self.spatial_attn1(e3)
        e3_att = e3
        d3 = self.dec3(torch.cat([u3, e3_att], dim=1))

        u2 = self.up2(d3)
        #e2_att = self.spatial_attn2(e2)
        e2_att = e2
        d2 = self.dec2(torch.cat([u2, e2_att], dim=1))

        u1 = self.up1(d2)
        #e1_att = self.spatial_attn3(e1)
        e1_att = e1
        d1 = self.dec1(torch.cat([u1, e1_att], dim=1))

        out = torch.sigmoid(self.final(d1))

        return out



In [123]:


# -----------------------
# PatchGAN Discriminator (70x70-ish style)
# -----------------------
class PatchDiscriminator(nn.Module):
    def __init__(self, in_ch=2, base_ch=64, n_layers=4):
        """
        in_ch: 入力チャネル（通常は concat(rough, line) -> 2ch, もしくは 1ch + 1ch = 2）
        base_ch: 最初のチャネル数
        n_layers: 畳み込み段数（空間縮小回数）
        """
        super().__init__()
        layers = []
        # Conv 1: no norm, leakyrelu
        layers.append(nn.Conv2d(in_ch, base_ch, kernel_size=4, stride=2, padding=1))
        layers.append(nn.LeakyReLU(0.2, inplace=True))

        ch = base_ch
        # middle layers
        for i in range(1, n_layers):
            in_c = ch
            out_c = min(ch * 2, 512)
            stride = 2 if i < n_layers-1 else 1  # 最後の畳み込みは stride=1 にして patch 出力の解像度を保つ
            layers.append(nn.Conv2d(in_c, out_c, kernel_size=4, stride=stride, padding=1, bias=False))
            layers.append(nn.InstanceNorm2d(out_c, affine=True))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            ch = out_c

        # final conv -> 1 channel output (patch map)
        layers.append(nn.Conv2d(ch, 1, kernel_size=4, stride=1, padding=1))
        # NOTE: no sigmoid here for LSGAN (we'll use MSELoss directly)
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)  # shape B x 1 x H_patch x W_patch

# -----------------------
# Replay (History) Buffer
# -----------------------
class ReplayBuffer:
    def __init__(self, max_size=50):
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, items):
        """
        items: tensor (B, C, H, W)
        returns: tensor (B, C, H, W) where some items may be replaced by older ones
        """
        returned = []
        for item in items:
            item = torch.unsqueeze(item.data, 0)
            if len(self.data) < self.max_size:
                # fill buffer
                self.data.append(item)
                returned.append(item)
            else:
                if random.random() > 0.5:
                    # use a previously stored one
                    idx = random.randint(0, self.max_size - 1)
                    tmp = self.data[idx].clone()
                    self.data[idx] = item
                    returned.append(tmp)
                else:
                    # use current
                    returned.append(item)
        return torch.cat(returned, dim=0)


# -----------------------
# Training loop with PatchGAN + LSGAN
# -----------------------
def train_with_gan(
    rough_dir='dataset/train/rough',
    line_dir='dataset/train/line',
    epochs=50,
    batch_size=8,
    lr_g=1e-4,
    lr_d=4e-4,
    device='cuda',
    use_history_buffer=True
):
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    dataset = PairImageDataset(rough_dir, line_dir)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

    # models
    G = AttnResUNet(in_ch=1, out_ch=1, base_ch=64, use_self_attn=False).to(device)
    D = PatchDiscriminator(in_ch=2, base_ch=64, n_layers=3).to(device)

    # losses
    mse_loss = nn.MSELoss()      # LSGAN
    l1_loss = nn.L1Loss()
    edge_loss = SobelEdgeLoss()

    # optimizers
    opt_G = torch.optim.AdamW(G.parameters(), lr=lr_g, betas=(0.5, 0.999), weight_decay=1e-4)
    opt_D = torch.optim.AdamW(D.parameters(), lr=lr_d, betas=(0.5, 0.999), weight_decay=1e-4)

    # schedulers (optional)
    scheduler_G = torch.optim.lr_scheduler.CosineAnnealingLR(opt_G, T_max=epochs*len(loader))
    scheduler_D = torch.optim.lr_scheduler.CosineAnnealingLR(opt_D, T_max=epochs*len(loader))

    # mixed precision
    scaler_G = torch.cuda.amp.GradScaler()
    scaler_D = torch.cuda.amp.GradScaler()

    # history buffer for fake images
    replay_buffer = ReplayBuffer(max_size=50) if use_history_buffer else None

    save_dir = 'checkpoints_gan'
    os.makedirs(save_dir, exist_ok=True)

    # weights
    w_l1 = 5
    w_edge = 10
    w_gan = 5  # weight for adversarial loss (generator)

    # labels for LSGAN
    real_label_value = 1.0
    fake_label_value = 0.0

    for epoch in range(1, epochs+1):
        G.train()
        D.train()
        running_loss = 0.0

        for i, (rough, line) in enumerate(loader):
            rough = rough.to(device, non_blocking=True)
            line = line.to(device, non_blocking=True)

            # --------------------
            # 1) Update D: maximize L_lsgan(D) = 0.5 * E[(D(x_real)-1)^2] + 0.5 * E[(D(x_fake)-0)^2]
            # --------------------
            opt_D.zero_grad()
            with torch.cuda.amp.autocast():
                # real pair: concat(rough, line)
                real_input = torch.cat([rough, line], dim=1)  # B x 2 x H x W
                pred_real = D(real_input)
                # target for real: ones with same shape as pred_real
                valid = torch.full_like(pred_real, real_label_value, device=device)

                loss_D_real = mse_loss(pred_real, valid)

                # fake pair: concat(rough, G(rough))
                fake = G(rough).detach()  # detach so gradients aren't backprop to G
                fake_input = torch.cat([rough, fake], dim=1)

                # optionally use history buffer
                if replay_buffer is not None:
                    fake_to_D = replay_buffer.push_and_pop([f for f in fake])
                    # push_and_pop returns cpu/gpu tensors depending; ensure it's on device
                    fake_to_D = fake_to_D.to(device)
                    fake_input = torch.cat([rough, fake_to_D], dim=1)
                pred_fake = D(fake_input)
                fake_label = torch.full_like(pred_fake, fake_label_value, device=device)
                loss_D_fake = mse_loss(pred_fake, fake_label)

                loss_D = 0.5 * (loss_D_real + loss_D_fake)

            scaler_D.scale(loss_D).backward()
            scaler_D.step(opt_D)
            scaler_D.update()
            scheduler_D.step()

            # --------------------
            # 2) Update G: minimize adversarial + L1 + Edge
            # --------------------
            opt_G.zero_grad()
            with torch.cuda.amp.autocast():
                gen = G(rough)
                # adversarial loss (want D(concat(rough, gen)) -> 1)
                pred_gen = D(torch.cat([rough, gen], dim=1))
                valid_for_gen = torch.full_like(pred_gen, real_label_value, device=device)
                loss_gan = mse_loss(pred_gen, valid_for_gen)

                # other losses
                loss_l1 = l1_loss(gen, line)
                loss_edge = edge_loss(gen, line)

                loss_G = w_gan * loss_gan + w_l1 * loss_l1 + w_edge * loss_edge

            scaler_G.scale(loss_G).backward()
            scaler_G.step(opt_G)
            scaler_G.update()
            scheduler_G.step()

            running_loss += loss_G.item()

            if (i+1) % 50 == 0:
                avg = running_loss / (i+1)
                print(f"Epoch[{epoch}/{epochs}] Iter[{i+1}/{len(loader)}] G_loss: {avg:.4f} (gan:{loss_gan.item():.4f} l1:{loss_l1.item():.4f} edge:{loss_edge.item():.4f})")

        avg_epoch = running_loss / len(loader)
        print(f"==> Epoch {epoch} finished. avg G loss: {avg_epoch:.4f}")

        # save checkpoint (both G and D)
        torch.save({
            'epoch': epoch,
            'G_state': G.state_dict(),
            'D_state': D.state_dict(),
            'optG': opt_G.state_dict(),
            'optD': opt_D.state_dict()
        }, os.path.join(save_dir, f'model_epoch_{epoch}.pth'))

    print("GAN Training finished.")


In [127]:
if __name__ == "__main__":
    # Example: point to your directories containing paired PNGs
    train_with_gan(
        rough_dir='dataset/train/rough',
        line_dir='dataset/train/line',
        epochs=18, # 時短のため30->18に変更
        batch_size=8,
        lr_g=2e-4,   # ← Generator の lr
        lr_d=0e-4,   # ← Discriminator の lr
        device='cuda'
    )

==> Epoch 1 finished. avg G loss: 6.4404
==> Epoch 2 finished. avg G loss: 5.1592
==> Epoch 3 finished. avg G loss: 4.7803
==> Epoch 4 finished. avg G loss: 4.5562
==> Epoch 5 finished. avg G loss: 4.4045
==> Epoch 6 finished. avg G loss: 4.3164
==> Epoch 7 finished. avg G loss: 4.2360
==> Epoch 8 finished. avg G loss: 4.1839
==> Epoch 9 finished. avg G loss: 4.1428
==> Epoch 10 finished. avg G loss: 4.1041
==> Epoch 11 finished. avg G loss: 4.0694
==> Epoch 12 finished. avg G loss: 4.0427
==> Epoch 13 finished. avg G loss: 4.0225
==> Epoch 14 finished. avg G loss: 4.0078
==> Epoch 15 finished. avg G loss: 3.9966
==> Epoch 16 finished. avg G loss: 3.9892
==> Epoch 17 finished. avg G loss: 3.9850
==> Epoch 18 finished. avg G loss: 3.9833
GAN Training finished.


In [125]:
# inference
def load_model(checkpoint_path, device='cuda'):
    model = AttnResUNet(in_ch=1, out_ch=1, base_ch=64, use_self_attn=True)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state'])
    model.to(device)
    model.eval()
    print(f"✅ Loaded checkpoint from {checkpoint_path}")
    return model

def preprocess_image(img_path, size=(256,256)):
    tr = transforms.Compose([
        transforms.Resize(size),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor()
    ])
    img = Image.open(img_path).convert('RGB')
    return tr(img).unsqueeze(0)  # 1x1xHxW tensor

def postprocess_and_save(tensor, save_path):
    # tensor: 1x1xHxW in [0,1]
    img = tensor.squeeze(0).squeeze(0).detach().cpu()  # HxW
    img = transforms.ToPILImage()(img)
    img.save(save_path)
    print(f"💾 Saved output to {save_path}")

@torch.no_grad()
def inference(model, input_path, output_path, device='cuda'):
    img = preprocess_image(input_path)
    img = img.to(device)

    with torch.amp.autocast(device_type='cuda'):
        output = model(img)

    postprocess_and_save(output, output_path)



In [126]:
if __name__ == "__main__":
    # ======== 設定 =========
    checkpoint_path = "checkpoints/model_epoch_18.pth"  # 学習済みモデル # 時短のため30->18に変更
    input_dir = "test/rough"   # テスト用の下絵
    output_dir = "results"             # 出力先
    os.makedirs(output_dir, exist_ok=True)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = load_model(checkpoint_path, device)

    # ======== 推論 =========
    rough_files = [f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    for fname in rough_files:
        in_path = os.path.join(input_dir, fname)
        out_path = os.path.join(output_dir, fname)
        inference(model, in_path, out_path, device)


✅ Loaded checkpoint from checkpoints/model_epoch_18.pth
💾 Saved output to results/sample.jpg
💾 Saved output to results/sample01.jpg


| パラメータ    | 効果                     | 備考                     |
| -------- | ---------------------- | ---------------------- |
| `w_l1`   | 全体の明るさや形の再現重視          | 通常 1.0 でOK             |
| `w_edge` | エッジ（線のシャープさ）をどれだけ重視するか | 大きいほど線がくっきり。ただしノイズも増える |


| 例      | 設定               | 効果の傾向                 |
| ------ | ---------------- | --------------------- |
| 柔らかめ   | `w_edge = 3.0`   | 線がややぼやけるがノイズ少ない       |
| 標準     | `w_edge = 5.0`   | バランス型（今の設定）           |
| シャープ強調 | `w_edge = 10.0`  | 線がくっきり、ややノイズ出る        |
| 強め     | `w_edge = 15〜20` | エッジ最重視、線は濃いがノイズが増える傾向 |


result
![result](./results/sample_w_edge6.jpg) 6
![result](./results/sample_w_edge8.jpg) 8
![result](./results/sample_w_edge10.jpg) 10
![result_w_edge20](./results/sample_w_edge20.jpg) 20


sample  
![sample](./test/rough/sample.jpg)

In [7]:
import onnx
def export_to_onnx(model, input_size, onnx_path="model.onnx"):
    # 1. CPUへ移動
    model = model.to("cpu")
    model.eval()

    # 2. 入力もCPU
    x = torch.randn(*input_size, device="cpu")

    torch.onnx.export(
        model, x, onnx_path,
        opset_version=13,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output']
    )
    print(f"[OK] Exported ONNX -> {onnx_path}")



In [8]:
model = load_model("checkpoints/model_epoch_20.pth")
export_to_onnx(model, (1,1,256,256))

✅ Loaded checkpoint from checkpoints/model_epoch_20.pth
[OK] Exported ONNX -> model.onnx


/home/sh1/deepl/mynovel/venv/lib/python3.10/site-packages/torch/onnx/symbolic_helper.py:1513: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'instance_norm' is set to train=True. Exporting with train=True.
  warnings.warn(


In [7]:
model = SelfAttention(512)
dummy = torch.randn(1, 512, 32, 32)

traced = torch.jit.trace(model, dummy)
traced.save("selfattention.pt")

![attn](./graph/selfattention.pt.svg)

🔍 この Self-Attention は何？

これは Non-Local Attention（Self-Attention） の典型実装で、
特徴マップ内の すべての位置同士の関連性を計算して、
「どこがどこと関係しているか」を学習します。

線画生成だと：

遠く離れた線の方向性

同じ構造の反復

長い輪郭線のつながり

などを補完できるので U-Net によく使われます。

Self-Attention（Non-Local Block）を U-Net に追加した場合、実質的に “新しいハイパーパラメータはほぼ生まれません。”
ただし、設置位置・数・in_dim（チャネル）・学習率への影響 という “準ハイパーパラメータ” は存在します。 

★ ユーザーが調整する必要がある（準ハイパーパラメータ）

どの層に入れるか（最重要）

何個入れるか

学習率を変えるか（場合により）

in_dim（チャネル数）の合う層を選ぶ必要

★ ユーザーが調整しなくてよい（ハイパーパラメータでない）

γ

attention の温度

head 数

dropout

kernel size

scaling factor

これらはすべて自動で機能するので触らない。

In [6]:
model = AttnResUNet()
dummy = torch.randn(1,1,256,256)

out = model(dummy)
dot = make_dot(out, params=dict(model.named_parameters()))
dot.render("attnres_graph", format="png")

'attnres_graph.png'

![attnres](./graph/attnres_graph.png)